In [1]:
import ast
import random

import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('../output_data/detailed_travel_routes.csv')

# Parse columns
df['Domain'] = df['Domain'].apply(lambda x: ast.literal_eval(x.replace('coordinates = ', '').strip()))
df['Plan'] = df['Plan'].apply(ast.literal_eval)
df['Detailed Plan'] = df['Detailed Plan'].apply(ast.literal_eval)

df

,id,Domain,Goal,Plan,Detailed Plan
0,0,"[[-71.22407723, 42.20650415], [-71.06341724278...",WESTWOOD/PEMBROKE HEALTH SYSTEM WESTWOOD (WEST...,"[[Drive north on Abbey Road., Turn left onto C...",[[{'instruction': 'Drive north on Abbey Road.'...
1,1,"[[-80.31080036, 26.01301541], [-80.21063654, 2...","MEMORIAL HOSPITAL WEST (PEMBROKE PINES, FL) to...","[[Drive northwest., Turn left after 19 meters ...","[[{'instruction': 'Drive northwest.', 'name': ..."
2,2,"[[-98.57619951, 29.52010675], [-97.7136084, 30...","CLARITY CHILD GUIDANCE CENTER (SAN ANTONIO, TX...","[[Drive east., Turn left onto Tom Slick., Turn...","[[{'instruction': 'Drive east.', 'name': '-', ..."
3,3,"[[-81.4371032715, 28.2898006439], [-80.3951633...","Kissimmee Gateway Airport (ORLANDO, FL) to IND...","[[Drive east on Patrick Street., Turn left ont...",[[{'instruction': 'Drive east on Patrick Stree...
4,4,"[[-122.2488937, 38.11790375], [-122.8130035, 3...",ST. HELENA HOSPITAL CENTER FOR BEHAVIORAL HEAL...,"[[Drive north on Broadway Street., Turn left o...",[[{'instruction': 'Drive north on Broadway Str...
...,...,...,...,...,...
495,495,"[[-90.709503, 42.402], [-92.40029907226562, 42...","Dubuque Regional Airport (DUBUQUE, IA) to Wate...","[[Drive north., Bear right after 1129 meters o...","[[{'instruction': 'Drive north.', 'name': '-',..."
496,496,"[[-76.56655563780009, 42.9295192], [-76.106300...","Seward, William H., House (Auburn, NY) to Syra...","[[Drive north., Turn left onto South Street/NY...","[[{'instruction': 'Drive north.', 'name': '-',..."
497,497,"[[-71.06823934295548, 42.35671345], [-72.2428,...","Headquarters House (Boston, MA) to BAYSTATE MA...","[[Drive west on Beacon Street., Bear left towa...",[[{'instruction': 'Drive west on Beacon Street...
498,498,"[[-86.58465712669839, 34.73047047522626], [-87...","Milligan Block (Huntsville, AL) to Northwest A...","[[Drive northwest on East Side Square., Take t...",[[{'instruction': 'Drive northwest on East Sid...


#### Choose steps and add questions and ground truth

In [2]:

df = df.sample(n=100, random_state=42)

df['Plan'] = df['Plan'].apply(lambda p: p[0])

random.seed(42)

def get_question_and_ground_truth(row):
    step_idx = random.randint(0, len(row['Plan']) - 2)

    return (f"Can Step {step_idx + 2} be performed without completing Step {step_idx +1}?", "No")

df[['Question', 'Ground Truth']] = df.apply(get_question_and_ground_truth, axis=1, result_type="expand")

#### Sort and save data

In [3]:
df = df[[
    'id',
    'Goal',
    'Plan',
    'Question',
    'Ground Truth',
]]

df = df.sort_values(by=['id'])

df.to_csv('../output_data/data-flow_analyses/type_state_analysis.csv', index=False)

df

,id,Goal,Plan,Question,Ground Truth
0,0,WESTWOOD/PEMBROKE HEALTH SYSTEM WESTWOOD (WEST...,"[Drive north on Abbey Road., Turn left onto Cl...",Can Step 10 be performed without completing St...,No
2,2,"CLARITY CHILD GUIDANCE CENTER (SAN ANTONIO, TX...","[Drive east., Turn left onto Tom Slick., Turn ...",Can Step 14 be performed without completing St...,No
9,9,"MBS International Airport (FREELAND, MI) to Be...","[Drive north on Flight Service Drive., Turn ri...",Can Step 2 be performed without completing Ste...,No
11,11,"Tuscaloosa National Airport (TUSCALOOSA, AL) t...","[Drive northeast., Turn left after 136 meters ...",Can Step 6 be performed without completing Ste...,No
15,15,"WHITTIER PAVILION (HAVERHILL, MA) to New Bedfo...","[Drive north., Turn left after 44 meters or 7-...",Can Step 8 be performed without completing Ste...,No
...,...,...,...,...,...
490,490,"Merrick-Simmons House (Fernandina Beach, FL) t...","[Drive north on South 10th Street., Turn left ...",Can Step 5 be performed without completing Ste...,No
491,491,"MILFORD HOSPITAL (MILFORD, CT) to Groton New L...","[Drive southeast., Turn left after 36 meters o...",Can Step 16 be performed without completing St...,No
494,494,NELL J. REDFIELD MEMORIAL HOSPITAL (MALAD CITY...,"[Drive south., Turn left onto Bannock Street.,...",Can Step 6 be performed without completing Ste...,No
495,495,"Dubuque Regional Airport (DUBUQUE, IA) to Wate...","[Drive north., Bear right after 1129 meters or...",Can Step 2 be performed without completing Ste...,No
